<center><h1><b>Proyek Analisis Sentimen</b></h1></center>
<center><h2><b>Sentimen Review Sociolla</b></h2></center>
<center><h3><b>Created by: Lis Wahyuni</b></h3></center>

In [1]:
# Load the saved model and necessary components
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/liswahyuni/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Load model and components
best_model = load_model('best_model.h5')

# Load tokenizer
with open('tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load label encoder
with open('label_encoder.pkl', 'rb') as handle:
    le1 = pickle.load(handle)

In [3]:
# Stopwords
stop_words = stopwords.words('indonesian')

In [4]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    # Split into words
    words = text.split()
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

In [5]:
def predict_sentiment(text, model, tokenizer, label_encoder):
    # Preprocess text
    text = preprocess_text(text)
    
    # Tokenize and pad
    sequence = tokenizer.texts_to_sequences([text])
    MAX_LENGTH = 20
    padded = pad_sequences(sequence, maxlen=MAX_LENGTH, padding='post')
    
    # Predict
    prediction = model.predict(padded)
    predicted_class = label_encoder.inverse_transform([prediction.argmax()])[0]
    
    # Get probabilities
    class_probabilities = {label_encoder.inverse_transform([i])[0]: prediction[0][i] 
                         for i in range(len(prediction[0]))}
    
    return predicted_class, class_probabilities

In [6]:
le1.classes_

array(['negative', 'neutral', 'positive'], dtype=object)

In [14]:
# Test with explicit examples
test_reviews = [
    "beh aloe vera toner benton sih bagus banget pas autumn winter tinggal negri lembab bgt wajahku kenyal gitu karna emg kulit tuh acne prone skin and combination kalo indo cocok pakenya malem yah",
    "kemasannya jar label warna pink imut nggak suka teksturnya heavy membentuk lapisan kulit meresap pas udah kering tuh kulit dipegang lapisan moisturizer rontokrontok daki layeringnya bingung sih ditimpa face oil sleeping mask sulit lapisannya rontokrontok aromanya nggak suka ga suka wangi lavender intens sih aromanya",
    "kasih review guys launching udh exited produknya wardah biru udh oranye btw"
]

print("Detailed Analysis Results:\n")
for text in test_reviews:
    # Show each step of processing
    print(f"Original Text: {text}")
    processed_text = preprocess_text(text)
    print(f"After preprocessing: {processed_text}")
    
    # Show tokenization result
    sequence = tokenizer.texts_to_sequences([processed_text])
    print(f"Tokenized: {sequence}")
    
    # Show prediction details
    padded = pad_sequences(sequence, maxlen=20, padding='post')
    raw_prediction = best_model.predict(padded)
    print(f"Raw prediction values: {raw_prediction[0]}")
    
    # Final prediction
    sentiment, probabilities = predict_sentiment(text, best_model, tokenizer, le1)
    print(f"Predicted Sentiment: {sentiment.upper()}")
    print("Confidence Scores:")
    for class_name, prob in probabilities.items():
        print(f"- {class_name}: {prob:.2%}")
    print("-" * 80 + "\n")

Detailed Analysis Results:

Original Text: beh aloe vera toner benton sih bagus banget pas autumn winter tinggal negri lembab bgt wajahku kenyal gitu karna emg kulit tuh acne prone skin and combination kalo indo cocok pakenya malem yah
After preprocessing: beh aloe vera toner benton sih bagus banget pas autumn winter tinggal negri lembab bgt wajahku kenyal gitu karna emg kulit tuh acne prone skin and combination kalo indo cocok pakenya malem yah
Tokenized: [[1, 129, 540, 3, 126, 10, 17, 12, 26, 1, 3217, 206, 13747, 35, 19, 182, 197, 22, 47, 274, 2, 80, 84, 231, 13, 34, 552, 24, 3765, 8, 243, 292, 558]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Raw prediction values: [3.4856144e-05 2.0376385e-05 9.9994481e-01]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted Sentiment: POSITIVE
Confidence Scores:
- negative: 0.00%
- neutral: 0.00%
- positive: 99.99%
--------------------------------------------------------------------------------

Original Text: kemasannya jar label warna pink imut nggak suka